In [1]:
import pandas as pd
import numpy  as np
import os
import re
import datetime
import pytz
import copy
from collections import Counter
from emoji   import UNICODE_EMOJI
from functools   import reduce
import sys
sys.path.append('/home/handeully/')
import bigquery_etl as bq
import operator
import time
from IPython.display import display
# NLP Env.
import nltk
from nltk import FreqDist
from nltk.corpus   import stopwords
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer,LancasterStemmer
from nltk.corpus   import wordnet
from nltk.corpus   import sentiwordnet as swn
from nltk import sent_tokenize,word_tokenize,pos_tag

# stop_words = stopwords.words('english')
import spacy
import gensim
from gensim import corpora
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import ray
import psutil
from tqdm.notebook import tqdm

#glove
from glove import Corpus,Glove

# Vis Env.
import pyLDAvis
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# GCP Env.
import google.auth
from google.cloud import bigquery
from googletrans import Translator
from google_trans_new import google_translator

# Coding Env.
import warnings
from pandas.core.common import SettingWithCopyWarning

credentials, project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

client = bigquery.Client(credentials=credentials, project=project_id )
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import tensorflow_hub as hub
import tensorflow as tf


#initialization ray
num_logit_cpus = psutil.cpu_count()
print(f'multiprocessing using {num_logit_cpus} cores')
ray.init(ignore_reinit_error=True,num_cpus=num_logit_cpus)



multiprocessing using 16 cores


2021-06-21 08:02:46,469	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8267


{'node_ip_address': '10.178.0.34',
 'raylet_ip_address': '10.178.0.34',
 'redis_address': '10.178.0.34:45140',
 'object_store_address': '/tmp/ray/session_2021-06-21_08-02-45_855173_3012/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-06-21_08-02-45_855173_3012/sockets/raylet',
 'webui_url': '127.0.0.1:8267',
 'session_dir': '/tmp/ray/session_2021-06-21_08-02-45_855173_3012',
 'metrics_export_port': 58871,
 'node_id': '0786f1f6a92b85f1e4159bc7e64d79dab706d79e1cb315996226e22b'}

In [2]:

elmo = hub.load('https://tfhub.dev/google/elmo/2')
# elmo = hub.Module("https://tfhub.dev/google/elmo/2", name="ELMO", trainable=True)

In [3]:
tensor_of_strings = tf.constant(["way too soft ast",
                                 "it is too soft awful",
                                 "Lazy"])
a = elmo.signatures['default'](tensor_of_strings)['word_emb'][0]
b = elmo.signatures['default'](tensor_of_strings)['word_emb'][1]

def cos_sim(a, b):
    return np.inner(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))


print("ELMo:", cos_sim(a,b))


ELMo: [[ 9.5180012e-03 -8.0246813e-05  2.6493142e-03 -5.5266230e-04
  -1.3538116e-02]
 [ 7.7355630e-03  2.7226850e-03  2.8787476e-01  2.4287330e-02
   4.1685767e-02]
 [-1.5542916e-02 -8.7496545e-03  2.4287330e-02  2.7392098e-01
   4.8746392e-02]
 [-1.1603009e-03 -3.7374350e-03  3.0211162e-02  8.4097041e-03
   2.9106410e-02]
 [ 1.2297216e-03  3.5184834e-03  5.8938339e-03  1.9753056e-04
  -2.9416273e-03]]


In [4]:
elmo.signatures['default'](tensor_of_strings)['default']

<tf.Tensor: shape=(3, 1024), dtype=float32, numpy=
array([[ 0.1356267 , -0.34732044,  0.40111545, ...,  0.06623798,
         0.0930934 ,  0.21503545],
       [-0.21489973, -0.41656438,  0.14316607, ..., -0.12296386,
         0.42859054,  0.34723777],
       [-0.22628236, -0.5580681 ,  0.18499218, ...,  0.30330458,
        -0.1853257 ,  0.37362975]], dtype=float32)>

In [5]:
elmo.signatures['default'](tensor_of_strings)['word_emb']

<tf.Tensor: shape=(3, 5, 512), dtype=float32, numpy=
array([[[ 3.0639046e-01, -1.8356988e-01, -1.4832385e-01, ...,
          4.9684113e-01,  2.2097029e-01,  3.6310643e-02],
        [-9.0264246e-02, -7.2146845e-01, -4.7797453e-01, ...,
          4.1464072e-01,  3.0869979e-01, -3.6598146e-02],
        [ 9.6202493e-03, -6.1759092e-02,  4.6186161e-01, ...,
          4.8391622e-01,  5.8916688e-01,  5.6873739e-01],
        [ 5.5276489e-01,  3.2464862e-01,  2.5802833e-01, ...,
         -3.3126295e-01, -6.3303626e-01,  4.7354150e-01],
        [-8.5225224e-02, -1.3059647e-01,  1.2390486e-01, ...,
          7.7495039e-02, -4.2895086e-02, -4.9512655e-02]],

       [[ 1.0388984e-01, -4.5646822e-01,  6.0496852e-02, ...,
         -4.3939475e-02, -7.4158609e-04, -4.3987298e-01],
        [ 1.9154720e-01,  2.2998667e-01, -2.8943861e-01, ...,
         -6.4465389e-02,  5.8101863e-01,  2.1768014e-01],
        [-9.0264246e-02, -7.2146845e-01, -4.7797453e-01, ...,
          4.1464072e-01,  3.0869979e-01, -3